Part 2 of **time series forecasting with energy**

In this section, I will investigate the problem at hand, formalising it, outlining methods for interpreting error, and setup a baseline using standard statistical methods for time-series analysis. The library `statsmodels` will come in handy with this task.

In [0]:
# LOAD THE REPOSITORY
# if you are working from outside the repository
# this happens if you use colab like I do
!git clone https://github.com/sandeshbhatjr/energy-prediction.git
!pip install -U --quiet pandas statsmodels

In [0]:
import datetime as dt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Predictive models

This part will be technical. The purpose is to explore SOTA models for time-series forecasting, and their applicability to day-ahead price forecasting. In a typical use-case, we have access to a history of recorded day-ahead prices, and we are interested in forecasting the day-ahead prices in an interval consisting of a tuple of 24 prices for each day. The problem can be formulated precisely as follows.
>  Let $\{t_1, t_2, t_3, ..., t_k \} := \mathfrak{T}$ represent a collection of dates in  chronological order, $p_t \in \mathbb{R}^{24}$ be the day-ahead price indexed by $t \in \mathfrak{T}$. Given some historic day-ahead prices $p_{t_1}, p_{t_2}, ... ,p_{t_k}$, forecast the future day-ahead prices $p_{t_{k+1}}, ..., p_{t_{k+h}}$ in a certain future window-size of $h \in \mathbb{N}$. In addition, we have a collection of exogenous time series- $d^{\alpha}_{t_1},d^{\alpha}_{t_2}, ..., d^{\alpha}_{t_k}$ indexed by $\alpha$, which includes the generation and consumption forecasts. Finally for each $t\in {t_1, ..., t_{k+h'}}, h'>> h$, we have some associated features $f^{\beta}_t$, where the features are known well into future (for example, if the day is a holiday or not).

In [0]:
def extractHourlyData(df, data_columns, hour_column='Hour', date_column='Date'):
  num_of_sample = len(df['Date'].unique())
  num_of_data_columns = len(data_columns)
  hourly_data = np.zeros((num_of_sample, 24*num_of_data_columns))
  for i, data_column in enumerate(data_columns):
    for d in range(24):
      hourly_data[:, i*24 + d] = df[df[hour_column] == d].sort_values(by=date_column).loc[:, data_column]
  return hourly_data

**Windowed dataset using Keras Timeseries generator**

Most models will work not on the entire time-series, but on a windowed subset of the time series. Keras has an inbuilt windowed dataset generator, but it helps to know a few things before we use that. It is primarily meant for use with Keras, so it produces a generator that outputs the dataset in batches as a list $[b_1, b_2, ...]$. Each batch $b_i$ consists of $[X,y]$, with each being a numpy array. The format here is as follows: each $X$ is a windowed dataset with following indices: `(sample_num, timestep, feature)`, while $y$ is of the form `(sample_num, value)`. The assumption here is that `y` consists of a single timestep, which works fine in our case.

In [0]:
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

def get_windowed_dataset(df, X_cols, y_cols, window_size, return_generator=False, **kwargs):
  X_data = extractHourlyData(df, X_cols)
  y_data = extractHourlyData(df, y_cols)
  # use batch size if defined, else return the whole dataset in one batch
  batch_size = kwargs['batch_size'] or len(X_data)
  generator = TimeseriesGenerator(
    X_data, 
    y_data, 
    length = window_size, 
    sampling_rate = 1,
    batch_size = batch_size)
  if return_generator:
    return generator
  else:
    X_batches = [X for (X, y) in generator]
    y_batches = [y for (X, y) in generator]
    return X_batches, y_batches

Just a quick check to see if the generator works as expected:

In [0]:
generator = get_windowed_dataset(
    german_df, 
    ['Day Ahead Price'], 
    ['Day Ahead Price'], 
    1, 
    return_generator=True, 
    batch_size=len(german_df)
)

generator[0][0].shape, generator[0][1].shape

Before proceeding to investigate our models, it is important to figure out a cross-validation strategy and testing strategy using a metric to assess the performance of each of our models. So, let us briefly touch upon these topics first.  

1. The cross-validation strategy for temporal data is a bit different from randomly choosing a subset of the dataset. Since the goal of our analysis is to forecast future values using present data, we will likewise train on a dataset from the past, and validate it against a subset to the future of it. This simulates the actual situation at hand; see [3, 4] for more details.
2. For a simple criterion to do a quick comparison of algorithms as a first step in model evaluation, the sMAPE suffices:
$$s = \frac{|y-\hat{y}|}{2(|y| + |\hat{y}|)} $$ This should be sufficient for a first-order estimate of how well our models are doing; later on, we can perform more sophisticated forms of tests for our analysis.

To cover pt. 1, we create a function to extract dataset into a train and test form chronologically separated as described. We also create a GridsearchCV equivalent for time series for hyperparameter search in ML models.

In [0]:
def test_train_timesplit(df, date_col_name='Date', train_size=0.9, test_size=0.1):
  """
    Returns test-train split data based on date with test chronologically later than train data.
  """
  min_date = df[date_col_name].min()
  max_date = df[date_col_name].max()
  train_split_date = min_date + (train_size*(max_date - min_date))
  test_split_date = train_split_date + (test_size*(max_date - min_date))
  train_df = df[df[date_col_name] < train_split_date]
  test_df = df[(df[date_col_name] > train_split_date) & (df[date_col_name] < test_split_date)]
  return train_df, test_df

def day_forward_chaining(df, date_col_name='Date', k=10):
  for i in range(1,k):
    yield test_train_timesplit(df, date_col_name, train_size=(i/k), test_size=(1/k))

# The above function is the time-series equivalent of gridsearch CV
# Example use-case:
# for train_df, test_df in day_forward_chaining(german_df):
#   <<< do your model training and tuning here >>>

# Naive model and the baseline

The simplest model one can think of is as follows: $p_{t+h} = p_t $ for all $h$ in the future window. This should allow us to setup a baseline, which any sensible model should perform better than. This is primarily helpful in debugging complex models when making sure that we are not doing anything terribly stupid.

# Regression

For time-series data, regression usually presents itself in the form of auto-regression, i.e. regression in terms of previous time-step variable. A very simple example would be the $AR(k)$ models, where the hypothesis is as follows:
$$p_{t+h}=\sum_{i=0}\alpha_ip_{t-i}+\epsilon$$ where $\epsilon$ is random white noise.

The above technique is a very simple one; there are far more sophistical statistical techniques available. We will pursue them here, but will use the `statsmodels` library. I will not explore the algorithms itself in detail (though it will make for a good series of blog posts sometime in the near future; for now refer to [1] for details).

## Univariate AR(k) with time trend

In [0]:
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics

AR_model = AutoReg(german_df, 3, trend='t')

## Multivariate VAR(k)

# Exponential Smoothing and Holtz-Winter models

# Markov switching regression

# Bibliography

**[1]**. R. J. Hyndman, and G. Athanasopoulos, Forecasting: Principles and Practice, https://otexts.com/fpp2/index.html 